In [356]:
import random
import copy

class TeekoPlayer:
    """ An object representation for an AI game player for the game Teeko.
    """
    board = [[' ' for j in range(5)] for i in range(5)]
    pieces = ['b', 'r']

    def __init__(self):
        """ Initializes a TeekoPlayer object by randomly selecting red or black as its
        piece color.
        """
        self.my_piece = random.choice(self.pieces)
        self.opp = self.pieces[0] if self.my_piece == self.pieces[1] else self.pieces[1]
        self.phase = ' '
        self.player = ' '
        self.total_moves = 0

    def determine_phase(self, board_state):
        num_r = 0
        num_b = 0
        for row in board_state:
            print(row)
            if 'r' in row:
                num_r += 1
            if 'b' in row:
                num_b += 1

        # determine Phase
        if num_r + num_b < 8:
            self.phase = "DROP"
        elif num_r + num_b == 8:
            self.phase = "MOVE"

        # determine Player
        if num_r + num_b == 0:
            self.player = 'b'
        elif num_r > num_b:
            self.player = 'b'
        elif num_r < num_b:
            self.player = 'r'
        elif self.total_moves % 2 == 0:
            self.player = 'b'
        elif self.total_moves % 2 != 0:
            self.player = 'r'

        return self.phase, self.player
    
    def get_opponent(self, piece):
        # Returns the opposing player's piece color given the current player's piece color.
        if piece == 'r':
            return 'b'
        elif piece == 'b':
            return 'r'
        else:
            raise ValueError('Invalid piece color: {}'.format(piece))
        
    def get_2x2_boxes(self, board, player):
        #Returns a list of all 2x2 boxes containing 3 pieces of the same player.
        boxes = []
        for i in range(4):
            for j in range(4):
                if board[i][j] == board[i+1][j] == board[i][j+1] == board[i+1][j+1] == player:
                    boxes.append((i, j))
        return boxes
    
    def get_adjacent_positions(self, row, col):
        #Given a position (row, col) on the Teeko board, returns a list of all adjacent positions.
        adjacent_positions = [(row-1, col), (row+1, col), #left, right
                              (row, col-1), (row, col+1), #Down, Up
                              (row-1, col-1), (row-1, col+1), # diagnals left
                              (row+1, col-1), (row+1, col+1)] # diagnals right
        return [(r, c) for r, c in adjacent_positions if 0 <= r < 5 and 0 <= c < 5]

    def succ(self, state, opponent = False):
        board = state
        phase, player = self.determine_phase(state)

        successors = []

        if opponent == True:
            player = self.get_opponent(player)

        # drop phase
        if phase == "DROP":
            for i in range(5):
                for j in range(5):
                    if board[i][j] == ' ':
                        new_board = [row[:] for row in board]
                        new_board[i][j] = player
                        new_phase = "MOVE" if len(self.get_2x2_boxes(new_board, player)) > 0 else "DROP"
                        successors.append((new_board, new_phase, self.get_opponent(player), (i,j)))
            return successors
        
        # move phase
        else:
            for i in range(5):
                for j in range(5):
                    if board[i][j] == player:
                        for ni, nj in self.get_adjacent_positions(i, j):
                            if board[ni][nj] == ' ':
                                new_board = [row[:] for row in board]
                                new_board[ni][nj] = player
                                new_board[i][j] = ' '
                                new_phase = "WIN" if self.is_win(new_board, player) else "MOVE"
                                successors.append((new_board, new_phase, self.get_opponent(player), (ni,nj)))
            return successors
        
    def game_value(self, state):
        """ Checks the current board status for a win condition

        Args:
        state (list of lists): either the current state of the game as saved in
            this TeekoPlayer object, or a generated successor state.

        Returns:
            int: 1 if this TeekoPlayer wins, -1 if the opponent wins, 0 if no winner

        TODO: complete checks for diagonal and box wins
        """

        # check horizontal wins
        for row in state:
            for i in range(2):
                if row[i] != ' ' and row[i] == row[i+1] == row[i+2] == row[i+3]:
                    return 1 if row[i]==self.my_piece else -1

        # check vertical wins
        for col in range(5):
            for i in range(2):
                if state[i][col] != ' ' and state[i][col] == state[i+1][col] == state[i+2][col] == state[i+3][col]:
                    return 1 if state[i][col]==self.my_piece else -1

        # check \ diagonal wins
        for i in range(2):
            if state[i][i] != ' ' and state[i][i] == state[i+1][i+1] == state[i+2][i+2] == state[i+3][i+3]:
                return 1 if state[i][i]==self.my_piece else -1

        # check / diagonal wins
        for i in range(2):
            j = 3 - i
            if state[i][j] != ' ' and state[i][j] == state[i+1][j-1] == state[i+2][j-2] == state[i+3][j-3]:
                return 1 if state[i][j]==self.my_piece else -1

        # check box wins
        for i in range(4):
            for j in range(4):
                if state[i][j] != ' ' and state[i][j] == state[i+1][j] == state[i][j+1] == state[i+1][j+1]:
                    return 1 if state[i][j]==self.my_piece else -1

        return 0 # no winner yet
    
    def heuristic_game_value(self, state):
        
        # Check if game is in terminal state
        value = self.game_value(state)
        if value != 0:
            return value

        # Evaluate non-terminal states heuristically
        # Number of pieces on the board
        num_pieces = 0
        for row in state:
            for col in row:
                if col != ' ':
                    num_pieces += 1
        piece_ratio = num_pieces / 16

        # Distance between pieces
        distances = []
        for i in range(5):
            for j in range(5):
                if state[i][j] == self.my_piece:
                    for k in range(i, 5):
                        for l in range(5):
                            if state[k][l] == self.my_piece:
                                distances.append(abs(i - k) + abs(j - l))
        avg_distance = sum(distances) / len(distances) if distances else 0

        # Combine the above factors into a heuristic value
        heuristic_value = piece_ratio * 0.4 + (1 - piece_ratio) * 0.6 * (1 - avg_distance / 4)

        return heuristic_value


    def generate_moves(self, state):
        """ Generates a list of all legal moves for the current player given the current state of the game.

        Args:
            state (list of lists): should be the current state of the game as saved in
                this TeekoPlayer object. Note that this is NOT assumed to be a copy of
                the board attribute.

        Returns:
            moves (list of tuples): a list of tuples, where each tuple represents a legal move. The first element
                of the tuple is a string representing the type of the move ('DROP' or 'MOVE'), and the second element
                is a tuple representing the coordinates of the move. If the move is of type 'DROP', the tuple contains
                only one element, which is the coordinates of the empty cell where the player drops its piece. If the move
                is of type 'MOVE', the tuple contains two elements: the coordinates of the cell from where the player moves
                its piece, and the coordinates of the cell where the player drops its piece.
        """
        phase, player = self.determine_phase(state)
        moves = []
        
        if phase == 'DROP':
            # Generate all legal DROP moves
            for i in range(5):
                for j in range(5):
                    if state[i][j] == ' ':
                        moves.append(('DROP', (i, j)))
        else:
            # Generate all legal MOVE moves
            for i in range(5):
                for j in range(5):
                    if state[i][j] == player:
                        for ni, nj in self.get_adjacent_positions(i, j):
                            if state[ni][nj] == ' ':
                                moves.append(('MOVE', ((i, j), (ni, nj))))
        return moves

    def make_move(self, state):
        """ Selects a (row, col) space for the next move. You may assume that whenever
        this function is called, it is this player's turn to move.

        Args:
            state (list of lists): should be the current state of the game as saved in
                this TeekoPlayer object. Note that this is NOT assumed to be a copy of
                the game state and should NOT be modified within this method (use
                place_piece() instead). Any modifications (e.g. to generate successors)
                should be done on a deep copy of the state.

                In the "drop phase", the state will contain less than 8 elements which
                are not ' ' (a single space character).

        Return:
            move (list): a list of move tuples such that its format is
                    [(row, col), (source_row, source_col)]
                where the (row, col) tuple is the location to place a piece and the
                optional (source_row, source_col) tuple contains the location of the
                piece the AI plans to relocate (for moves after the drop phase). In
                the drop phase, this list should contain ONLY THE FIRST tuple.

        Note that without drop phase behavior, the AI will just keep placing new markers
            and will eventually take over the board. This is not a valid strategy and
            will earn you no points.
        """
        self.total_moves += 1
        phase, player = self.determine_phase(state)

        drop_phase = True

        if phase == "MOVE":
            drop_phase = False   # TODO: detect drop phase

        if not drop_phase:
            # TODO: choose a piece to move and remove it from the board
            # (You may move this condition anywhere, just be sure to handle it)
            #
            # Until this part is implemented and the move list is updated
            # accordingly, the AI will not follow the rules after the drop phase!
            ai_pieces = []
            for row in range(5):
                for col in range(5):
                    if state[row][col] == self.my_piece:
                        ai_pieces.append((row, col))
            print(ai_pieces)
            depth = 3 # set depth to 3 for now
            moves = self.succ(state) # get all possible moves
            best_move = None
            best_score = float('-inf')
            for move in moves:
                score = self.MiniMax(move[0], depth, False)
                if score > best_score:
                    best_score = score
                    best_move = move[-1]

            for piece in ai_pieces:
                if best_move in self.get_adjacent_positions(piece):
                    state[piece[0]][piece[1]] = ' '

            return [best_move]

        # TODO: implement a minimax algorithm to play better

        move = []
        (row, col) = (random.randint(0,4), random.randint(0,4))
        while not state[row][col] == ' ':
            (row, col) = (random.randint(0,4), random.randint(0,4))

        # ensure the destination (row,col) tuple is at the beginning of the move list
        move.insert(0, (row, col))
        return move

    def MiniMax(self, state, d, max_player = True):
        phase, player = self.determine_phase(state)

        if self.game_value(state) == 1 or self.game_value(state) == -1:
            return self.game_value(state)
        elif d == 0:
            return self.heuristic_game_value(state)
        
        elif max_player:
            value = float('-inf')
            for successor in self.succ(state):
                value = max(value, self.MiniMax(successor[0], d-1, False))
                return value
        else:
            value = float('inf')
            for successor in self.succ(state):
                value = min(value, self.MiniMax(successor[0], d-1, True))
                return value



In [359]:
ai = TeekoPlayer()

board = ai.board

#phase, player = ai.determine_phase(board)

#ai.MiniMax(board, 3, True)
#ai.heuristic_game_value(board)
#ai.generate_moves(board)
ai.make_move(board)
#ai.succ(board)

[' ', ' ', ' ', ' ', ' ']
[' ', ' ', ' ', ' ', ' ']
[' ', ' ', ' ', ' ', ' ']
[' ', ' ', ' ', ' ', ' ']
[' ', ' ', ' ', ' ', ' ']


[(2, 0)]

In [ ]:
    '''def Max_Value(self, state, alpha=float('-inf'), beta=float('inf')):
        if self.game_value(state) == 1 or self.game_value(state) == -1:
            return self.game_value(state)
        else:
            for successor in self.succ(state):
                alpha = max(alpha, self.Min_Value(successor, alpha, beta))
                if alpha >= beta: # Alpha Pruning
                    return beta
                else:
                    return alpha
                
    def Min_Value(self, state, alpha=float('-inf'), beta=float('inf')):
        if self.game_value(state) == 1 or self.game_value(state) == -1:
            return self.game_value(state)
        else:
            for successor in self.succ(state):
                print(successor)
                beta = min(beta, self.Max_Value(successor, alpha, beta))
                if alpha >= beta: # Beta pruning
                    return alpha
                else:
                    return beta'''